In [55]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist
import os
import networkx as nx
from pyvis import network as net
import random

In [20]:
x = pd.read_csv("/scratch/jho5ze/bionets/genome_metadata/data/meta/genome_lineage", \
            sep="\t", dtype={"genome_id":str, "taxon_id":str})
# x[x["genome_id"]]
x[x["genome_id"] == "469009.4"].iloc[0].to_dict()

{'genome_id': '469009.4',
 'genome_name': '"\'Brassica napus\' phytoplasma strain TW1"',
 'taxon_id': '469009',
 'kingdom': 'Bacteria',
 'phylum': 'Tenericutes',
 'class': 'Mollicutes',
 'order': 'Acholeplasmatales',
 'family': 'Acholeplasmataceae',
 'genus': 'Candidatus Phytoplasma',
 'species': "'Brassica napus' phytoplasma",
 'taxon_lineage_ids': '131567;2;1783272;544448;31969;186329;2146;33926;85620;469009',
 'taxon_lineage_names': "cellular organisms;Bacteria;Terrabacteria group;Tenericutes;Mollicutes;Acholeplasmatales;Acholeplasmataceae;Candidatus Phytoplasma;Candidatus Phytoplasma asteris;'Brassica napus' phytoplasma"}

In [23]:
# lineage_df = pd.read_csv("/scratch/jho5ze/bionets/genome_metadata/data/meta/genome_lineage", \
#                          sep="\t", dtype={"genome_id":str, "taxon_id":str})
# metadata_df = pd.read_csv("/scratch/jho5ze/bionets/genome_metadata/data/meta/genome_metadata", \
#                           sep="\t", dtype={"genome_id":str, "taxon_id":str})        
# summary_df = pd.read_csv("/scratch/jho5ze/bionets/genome_metadata/data/meta/genome_summary", \
#                          sep="\t", dtype={"genome_id":str, "taxon_id":str})
# amr_df = pd.read_csv("/scratch/jho5ze/bionets/genome_metadata/data/meta/PATRIC_genomes_AMR.txt", \
#                      sep="\t", dtype={"genome_id":str, "taxon_id":str})

/apps/software/standard/compiler/gcc/7.1.0/jupyter_conda/2019.10-py3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2,6,7,8,9,10,11,12,14,21,34,40,41,42,43,44,46,47,48,49,50,52,53,54,55,56,57,60,61,63) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/apps/software/standard/compiler/gcc/7.1.0/jupyter_conda/2019.10-py3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (4,5,6,7,8,10,11,12,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
"""
genome_id       genome_name     organism_name   taxon_id        genome_status   strain  serovar biovar   pathovar        mlst    other_typing    culture_collection      type_strain     completion_date  publication     bioproject_accession    biosample_accession     assembly_accession      genbank_accessions       refseq_accessions       sequencing_centers      sequencing_status       sequencing_platform      sequencing_depth        assembly_method chromosomes     plasmids        contigs  sequences       genome_length   gc_content      patric_cds      brc1_cds        refseq_cds       isolation_site  isolation_source        isolation_comments      collection_date isolation_country        geographic_location     latitude        longitude       altitude        depth   other_environmental      host_name       host_gender     host_age        host_health     body_sample_site body_sample_subsite     other_clinical  antimicrobial_resistance        antimicrobial_resistance_evidence        gram_stain      cell_shape      motility        sporulation     temperature_range        optimal_temperature     salinity        oxygen_requirement      habitat disease comments additional_metadata
"""

In [3]:
h = """
genome_id       genome_name     organism_name   taxon_id        genome_status   strain  serovar biovar   pathovar        mlst    other_typing    culture_collection      type_strain     completion_date  publication     bioproject_accession    biosample_accession     assembly_accession      genbank_accessions       refseq_accessions       sequencing_centers      sequencing_status       sequencing_platform      sequencing_depth        assembly_method chromosomes     plasmids        contigs  sequences       genome_length   gc_content      patric_cds      brc1_cds        refseq_cds       isolation_site  isolation_source        isolation_comments      collection_date isolation_country        geographic_location     latitude        longitude       altitude        depth   other_environmental      host_name       host_gender     host_age        host_health     body_sample_site body_sample_subsite     other_clinical  antimicrobial_resistance        antimicrobial_resistance_evidence        gram_stain      cell_shape      motility        sporulation     temperature_range        optimal_temperature     salinity        oxygen_requirement      habitat disease comments additional_metadata
"""
len(h.split())

66

In [47]:
# file = "../parsnp_test/parsnp.vcf"
file = "../data/genomes/test/patric_ecoli_100_out/parsnp.vcf"

with open(file) as src:
    line = src.readline()
    while line[:2] == "##":
        line = src.readline()
    splitter = "FORMAT" if "FORMAT" in line else "INFO"
    line = line.split()
    line[0] = line[0].split("#")[-1]
    org_ind = line.index(splitter) + 1
    orgs = [l.split(".fna")[0] for l in line[org_ind:]]
    num_orgs = len(orgs)
#     line = src.readline()
#     while line != "":
#         line = line.split()[org_ind:]

In [48]:
#Could also use np.fromfile with a little bit of shell setup, which would be more efficient
# vcf = pd.read_csv("../parsnp_test/parsnp.vcf", skiprows=5, sep="\t", names=line, header=None).head(6)
vcf = pd.read_csv(file, skiprows=5, sep="\t", names=line, header=None) #.head(6)
vcf = vcf.iloc[:, org_ind:]
entries = vcf.shape[0] - 1
distances = pdist(vcf.T.iloc[:, 1:].to_numpy(), metric="hamming") * entries

/home/jho5ze/.conda/envs/freeze/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (1,5,7,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [33]:
print(orgs)
print(max(distances))

['562.7843', '562.7514', '562.7539', '562.7163', '562.7509', '511145.12']
90139.0


In [44]:
metadata_df = pd.read_csv("/scratch/jho5ze/bionets/genome_metadata/data/meta/genome_metadata", \
                          sep="\t", dtype={"genome_id":str, "taxon_id":str})
metadata_df["sequencing_platform"].value_counts()

Illumina HiSeq                              47553
Illumina MiSeq                              35643
Illumina                                    33985
Illumina HiSeq 2000                         11949
Illumina HiSeq 2500                          8603
                                            ...  
PacBio RS II, Illumina GAIIx                    1
454 Titanium; Illumina GAII                     1
paired-end Illumina Miseq                       1
Illumina; Oxford Nanopore MinION                1
Illumina GAIIx; Illumina HiSeq; Illumina        1
Name: sequencing_platform, Length: 1703, dtype: int64

In [81]:
graph = nx.Graph()
metadata_df = pd.read_csv("/scratch/jho5ze/bionets/genome_metadata/data/meta/genome_metadata", \
                          sep="\t", dtype={"genome_id":str, "taxon_id":str})

random.seed(42)
snp_dist_thresh = 14000
color_attr = "sequencing_platform"
size_attr = "" #Not yet used
sequencing_platforms = ["Illumina HiSeq", "Illumina MiSeq", "Illumina", "Illumina HiSeq", "Illumina", "HiSeq"]

nodes = []
for org in orgs:
#     print(org)
    meta = metadata_df[metadata_df["genome_id"] == org].iloc[0].to_dict()
#     meta_str = "<br>".join(f"{k}: {v}" for k, v in meta.items())
#     node_attrs = {"group":meta[color_attr], "title":meta_str}
    random_platform = random.choice(sequencing_platforms)
    meta_str = "<br>".join([f"Genome: {org}", f"Sequencing Platform {random_platform}"])
    node_attrs = {"group":random_platform, "title":meta_str}
#     print("Group is: ", meta[color_attr])
#     metadata["group"] = metadata[color_attr]
#     node = {"genome_id":org, "metadata":meta}
#     print(org)
#     print(meta)
#     graph.add_node((org, meta))
    nodes.append((org, node_attrs))
    
graph.add_nodes_from(nodes)
counter = 0
for i in range(num_orgs-1):
    for j in range(1, num_orgs-i):
#         print(i)
#         print(j)
        to = orgs[i]
        fro = orgs[i+j]
#         print("From {} o {}".format(to, fro))        
#         print("The SNP dist is {}".format(distances[counter]))
        dist = distances[counter]
        counter += 1
        if dist < snp_dist_thresh:
#             print("Adding edge between {} and {} with distance {}".format(to, fro, dist))
            graph.add_edge(to, fro, value=1000000/(1+dist), title=f"Dist: {dist}")

In [82]:
g = net.Network(height="750px", width="100%", bgcolor="#222222", font_color="white", notebook=True)
g.from_nx(graph)
g.show("test.html")